# Spotify User Data Analysis

## Data Cleaning

Before I can conduct analysis on my user data, first I will clean the data, removing unnecessary columns and combining all the listening instances.

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [18]:
df = pd.read_csv("C:/Users/nyc8p/OneDrive/Desktop/spoty-records-master/output/final.csv")
#remove redundant columns from dataframe
df = df.drop(columns = ["name", "Unnamed: 0", "uri", "type", "id", "track_href", "analysis_url","albumID"], axis = 1)
df.head()

,endTime,artistName,trackName,msPlayed,datetime,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,albumName
0,2022-08-24 00:08,Kendrick Lamar,Swimming Pools (Drank) - Extended Version,313786,2022-08-24 00:08:00,0.716,0.485,1,-7.745,1,0.404,0.12300,0.000027,0.604,0.260,74.132,313787,4,"good kid, m.A.A.d city (Deluxe)"
1,2022-08-24 00:13,Kendrick Lamar,Count Me Out,283642,2022-08-24 00:13:00,0.776,0.431,4,-7.544,0,0.091,0.67100,0.000000,0.153,0.495,133.999,283642,4,Mr. Morale & The Big Steppers
2,2022-08-24 00:16,Vince Staples,EAST POINT PRAYER,217855,2022-08-24 00:16:00,0.598,0.553,5,-8.911,1,0.344,0.07660,0.000000,0.364,0.201,77.779,217856,4,RAMONA PARK BROKE MY HEART
3,2022-08-24 00:26,Westside Gunn,Vogue cover (feat. Stove God Cooks),161280,2022-08-24 00:26:00,0.309,0.667,9,-6.557,1,0.141,0.00249,0.000022,0.148,0.482,148.319,161280,4,Hitler Wears Hermes 8: Sincerely Adolf
4,2022-08-24 00:29,Injury Reserve,Superman That,160062,2022-08-24 00:29:00,0.492,0.744,0,-9.371,1,0.359,0.24800,0.000009,0.236,0.533,110.142,160062,3,By the Time I Get to Phoenix


### Deleting the instances where I skipped a song

In [19]:
#drop all times I skipped a song, ie drop the row when msPlayed is under 15000
df = df[df['msPlayed'] >= 20000]
        

In [20]:
df

,endTime,artistName,trackName,msPlayed,datetime,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,albumName
0,2022-08-24 00:08,Kendrick Lamar,Swimming Pools (Drank) - Extended Version,313786,2022-08-24 00:08:00,0.716,0.485,1,-7.745,1,0.404,0.12300,0.000027,0.6040,0.260,74.132,313787,4,"good kid, m.A.A.d city (Deluxe)"
1,2022-08-24 00:13,Kendrick Lamar,Count Me Out,283642,2022-08-24 00:13:00,0.776,0.431,4,-7.544,0,0.091,0.67100,0.000000,0.1530,0.495,133.999,283642,4,Mr. Morale & The Big Steppers
2,2022-08-24 00:16,Vince Staples,EAST POINT PRAYER,217855,2022-08-24 00:16:00,0.598,0.553,5,-8.911,1,0.344,0.07660,0.000000,0.3640,0.201,77.779,217856,4,RAMONA PARK BROKE MY HEART
3,2022-08-24 00:26,Westside Gunn,Vogue cover (feat. Stove God Cooks),161280,2022-08-24 00:26:00,0.309,0.667,9,-6.557,1,0.141,0.00249,0.000022,0.1480,0.482,148.319,161280,4,Hitler Wears Hermes 8: Sincerely Adolf
4,2022-08-24 00:29,Injury Reserve,Superman That,160062,2022-08-24 00:29:00,0.492,0.744,0,-9.371,1,0.359,0.24800,0.000009,0.2360,0.533,110.142,160062,3,By the Time I Get to Phoenix
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28242,2023-08-23 17:40,Kanye West,Life Of The Party (with André 3000),391656,2023-08-23 17:40:00,0.430,0.717,2,-6.676,1,0.275,0.13600,0.000000,0.9720,0.302,78.881,391656,4,Life Of The Party
28243,2023-08-23 17:51,Roddy Ricch,Intro,135849,2023-08-23 17:51:00,0.647,0.715,0,-9.192,1,0.290,0.00599,0.000000,0.0662,0.649,141.936,148844,4,Tana Talk 3
28244,2023-08-23 17:52,JAY-Z,Empire State Of Mind,51255,2023-08-23 17:52:00,0.491,0.956,11,-1.538,1,0.392,0.02950,0.000000,0.4600,0.811,173.585,276920,4,The Blueprint 3
28246,2023-08-24 19:46,Dave,Location (feat. Burna Boy),241293,2023-08-24 19:46:00,0.812,0.496,9,-5.969,0,0.297,0.27100,0.000000,0.0955,0.550,109.979,241293,4,PSYCHODRAMA


### Combining Multiple Instances of Songs

Right now, the dataset is a collection of all the individual instances I played a song. Here, I combine all the entries with the same track name in order to add up the ms played for each song.

In [21]:
# Create a dictionary of aggregation functions
aggregation = {
    'msPlayed': 'sum',  # Sum 'minutes_played'
}

cols_to_keep = ['artistName','albumName', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
                'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
# Specify 'first' aggregation for all columns you want to keep as they are
for col in cols_to_keep:
    aggregation[col] = 'first'

In [22]:
#make function where group by song and then add the msPlayed
                    
df = df.groupby('trackName').agg(aggregation).reset_index()
df

,trackName,msPlayed,artistName,albumName,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,#FREEMIR,435280,Ghais Guevara,There Will Be No Super-Slave,0.454,0.878,7,-4.657,1,0.2090,0.157000,0.000000,0.432,0.335,142.790,164476,4
1,$1 One Vote!,179620,The The,$1 One Vote!,0.535,0.734,9,-8.389,1,0.0714,0.051500,0.005790,0.117,0.503,94.594,230436,4
2,(How Could Anybody) Feel at Home,191325,Open Mike Eagle,Brick Body Kids Still Daydream,0.623,0.819,7,-6.377,1,0.2220,0.121000,0.141000,0.118,0.705,79.435,254049,4
3,(Nice Dream),2414735,Radiohead,The Bends,0.263,0.566,9,-11.075,1,0.0344,0.163000,0.261000,0.225,0.375,172.851,233227,3
4,(Only) About Love - Demo,107868,grentperez,Demo(s) About Love,0.711,0.514,1,-10.565,0,0.0342,0.104000,0.120000,0.132,0.588,130.047,258000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3313,花傘,67730,indigo la End,濡れゆく私小説,0.681,0.822,5,-4.467,0,0.0355,0.078800,0.000000,0.367,0.561,125.887,280552,4
3314,遥か彼方,598426,ASIAN KUNG-FU GENERATION,BEST HIT AKG,0.308,0.955,4,-3.763,1,0.1110,0.000024,0.113000,0.166,0.237,174.991,243347,4
3315,雨降る夜の向こう,8175611,Lamp,ランプ幻想,0.542,0.436,7,-8.571,1,0.0273,0.805000,0.000007,0.096,0.511,73.505,311920,4
3316,飛行船,373200,Lily Chou-Chou,呼吸,0.493,0.977,9,-3.701,1,0.2030,0.082400,0.672000,0.317,0.237,112.919,186600,4


Now, let's convert msPlayed to minutes played to get a more readable column. While we're at it, let's get times played and duration in minutes as new columns as well.

In [23]:
df['minPlayed'] = df['msPlayed'] / 60000
df['timesPlayed'] = df['msPlayed'] / df['duration_ms']
df['duration_min'] = df['duration_ms'] / 60000
df = df.drop(columns = ['msPlayed'])

# First, create a list of column names in the desired order
new_column_order = list(df.columns)
new_column_order.remove('minPlayed')
new_column_order.insert(3, 'minPlayed')  # 3 represents the 4th position (0-based index)
new_column_order.remove('timesPlayed')
new_column_order.insert(4, 'timesPlayed')
new_column_order.remove('duration_min')
new_column_order.insert(5, 'duration_min')
# Reorder the columns in the DataFrame
df = df[new_column_order]

In [32]:
df.head()

,trackName,artistName,albumName,minPlayed,timesPlayed,duration_min,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,#FREEMIR,Ghais Guevara,There Will Be No Super-Slave,7.254667,2.646465,2.741267,0.454,0.878,7,-4.657,1,0.2090,0.157,0.000000,0.432,0.335,142.790,164476,4
3,(Nice Dream),Radiohead,The Bends,40.245583,10.353583,3.887117,0.263,0.566,9,-11.075,1,0.0344,0.163,0.261000,0.225,0.375,172.851,233227,3
6,...& On,Erykah Badu,Mama's Gun,31.317767,8.775228,3.568883,0.780,0.343,1,-12.130,1,0.0710,0.413,0.000154,0.307,0.724,83.417,214133,4
7,...And The World Laughs With You,Flying Lotus,Cosmogramma,2.917333,1.000000,2.917333,0.637,0.717,8,-7.635,0,0.1430,0.185,0.497000,0.141,0.541,80.926,175040,4
8,03’ Adolescence,J. Cole,2014 Forest Hills Drive,4.403550,1.000000,4.403550,0.528,0.811,1,-6.181,1,0.5900,0.395,0.000000,0.274,0.260,70.580,264213,4


It seems like I should filter some more, as there are some songs where timesPlayed is far below 1, meaning I skipped the song early on.

In [33]:
df = df[df['timesPlayed'] >= 1]

In [36]:
df = df.sort_values(["timesPlayed", 'minPlayed'], ascending = False)

In [37]:
df

,trackName,artistName,albumName,minPlayed,timesPlayed,duration_min,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
3026,Weird Fishes/ Arpeggi,Radiohead,In Rainbows,309.843417,58.426664,5.303117,0.531,0.6100,11,-8.025,0,0.0387,0.772,0.756000,0.0908,0.199,152.958,318187,4
1043,Good Will Hunting,"Black Country, New Road",Ants From Up There,238.539017,48.042983,4.965117,0.587,0.3580,2,-9.194,1,0.0276,0.219,0.002010,0.1920,0.368,96.685,297907,3
2106,"Pro Freak (with Doechii, Fatman Scoop)",Smino,Luv 4 Rent,202.782350,45.286325,4.477783,0.502,0.6410,1,-6.271,1,0.4670,0.269,0.000005,0.1970,0.212,75.008,268667,4
2098,Present Tense,Radiohead,A Moon Shaped Pool,229.408533,44.896820,5.109683,0.462,0.4070,1,-12.428,0,0.0345,0.912,0.399000,0.1100,0.336,91.915,306581,4
3264,the perfect pair,beabadoobee,Beatopia,131.329167,44.384706,2.958883,0.634,0.6630,11,-6.818,1,0.0331,0.433,0.124000,0.1020,0.600,146.053,177533,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1336,Is This Real? (Can You Hear Yourself?),Sudan Archives,Natural Brown Prom Queen,0.704667,1.000000,0.704667,0.286,0.2090,3,-12.302,0,0.0333,0.929,0.030300,0.1150,0.145,129.899,42280,4
1328,Intro (Inside My Mind),BJ The Chicago Kid,In My Mind,0.689550,1.000000,0.689550,0.556,0.2010,7,-20.726,1,0.4220,0.130,0.000001,0.5520,0.169,117.663,41373,4
770,Equally Damaged,Blonde Redhead,Melody of Certain Damaged Lemons,0.670000,1.000000,0.670000,0.528,0.0508,0,-22.086,0,0.1220,0.995,0.856000,0.1480,0.891,159.697,40200,3
2259,SCENE,BROCKHAMPTON,SATURATION II,0.658667,1.000000,0.658667,0.434,0.3120,5,-20.861,1,0.0567,0.936,0.001690,0.5530,0.862,145.864,39520,4


This dataset looks much better now, so now I will export it as another csv file!

In [38]:
df.to_csv("cleanhistory.csv")

# EDA

Now, we can take a look at the relationships between my most played songs and some of the features of those songs